<a href="https://colab.research.google.com/github/Gau-ravv/ML/blob/main/notebook8e8c4d9055.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

%pip install nltk
%pip install spacy
import nltk
nltk.download('punkt_tab')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download('stopwords')
stemmer = PorterStemmer()
Lemma = WordNetLemmatizer()
nltk.download('wordnet')
import spacy
import en_core_web_sm
import pandas as pd

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from tqdm import tqdm
import spacy
from nltk import word_tokenize, sent_tokenize
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner","tok2vec"])

In [ ]:
df2 = pd.read_csv("/kaggle/input/nlp-assign/pre_corpus.csv")  # Ensure df2 is the correct dataset
corpus = df2["Text"].dropna().tolist()

In [ ]:
df2

,Text
0,port lucie city lucie county florida united st...
1,dirty dozen refer books film television dirty ...
2,glenrothes town situated heart fife east centr...
3,roughnecks starship troopers chronicles cgi an...
4,events decisions edict milan signed constantin...
...,...
13874,northport historic maritime village northern s...
13875,ko knockout sports boxing martial arts ko kō r...
13876,daniel eugene rudy ruettiger born august motiv...
13877,chicago cubs american professional baseball te...


In [ ]:
df = pd.read_csv(r"/kaggle/input/nlp-assign/train(1).csv")
df_title = df.title

In [ ]:
%pip install torch
%pip install rouge_score

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

pre_corpus=[]
# # Load the English language model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner","tok2vec"])

texts = list(df_title)

# Process the texts
for doc in tqdm(nlp.pipe(texts, batch_size=50)):
    # Perform lemmatization and stopword removal
    lemmatized_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]

    # Join the tokens back into a string
    processed_text = " ".join(lemmatized_tokens)

    # Print the processed text
    pre_corpus.append(processed_text)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
13879it [00:02, 6925.38it/s]


In [ ]:
import pandas as pd


# Convert list to DataFrame
df = pd.DataFrame(pre_corpus, columns=["title"])

# Save as CSV
df.to_csv("pre_corpus_heading.csv", index=False)  # Set index=False to avoid unnecessary index column

print("CSV file saved successfully!")

CSV file saved successfully!


In [ ]:
df_title = df
df_title

,title
0,port lucie florida
1,dirty dozen
2,glenrothes
3,roughnecks starship troopers chronicles
4,june
...,...
13874,northport new york
13875,ko
13876,rudy ruettiger
13877,chicago cubs


In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

pre_corpus=[]
# # Load the English language model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner","tok2vec"])

texts = list(pd.read_csv(r"/kaggle/input/nlp-assign/test(1).csv").title)

# Process the texts
for doc in tqdm(nlp.pipe(texts, batch_size=50)):
    # Perform lemmatization and stopword removal
    lemmatized_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]

    # Join the tokens back into a string
    processed_text = " ".join(lemmatized_tokens)

    # Print the processed text
    pre_corpus.append(processed_text)

df_test_title = pd.DataFrame(pre_corpus, columns=["title"])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
100it [00:00, 4894.23it/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from rouge_score import rouge_scorer

# Define constants
MAX_NEW_TOKENS = 10
HIDDEN_DIM = 300
SOS_TOKEN = 0
EOS_TOKEN = 1
PAD_TOKEN = 2
UNK_TOKEN = 3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to create vocabulary
# def create_vocabulary(paragraphs, titles, min_frequency=0.01):
#     tokenized_words = [word for paragraph in paragraphs for word in paragraph.split()]
#     word_counts = Counter(tokenized_words)
#     total_texts = len(paragraphs)

#     vocabulary = {"<SOS>": SOS_TOKEN, "<EOS>": EOS_TOKEN, "<PAD>": PAD_TOKEN, "<UNK>": UNK_TOKEN}
#     vocabulary.update({
#         word: idx + 4 for idx, (word, count) in enumerate(word_counts.items()) if (count / total_texts) >= min_frequency
#     })
#     return vocabulary

from collections import Counter

def create_vocabulary(paragraphs, titles_train_raw,min_frequency=0.01):
    word_article_count = Counter()  # Track occurrences of words across different articles

    for paragraph in paragraphs:
        unique_words = set(paragraph.split())  # Unique words in each article
        word_article_count.update(unique_words)  # Count each word once per article

    total_articles = len(paragraphs)  # Total number of articles

    # Filter words that meet the minimum frequency criteria
    filtered_words = {word: count for word, count in word_article_count.items() if (count / total_articles) >= min_frequency}

    # Create vocabulary with only filtered words
    vocabulary = {"<SOS>": SOS_TOKEN, "<EOS>": EOS_TOKEN, "<PAD>": PAD_TOKEN, "<UNK>": UNK_TOKEN}
    vocabulary.update({word: idx + 4 for idx, word in enumerate(filtered_words)})

    # print(f"Number of words appearing in only one article: {sum(1 for count in filtered_words.values() if count == 1)}")
    print(f"Vocabulary size: {len(vocabulary)}")  # Debugging info

    return vocabulary


# Tokenization function
def tokenize_text(texts, vocabulary):
    return [[vocabulary.get(word, UNK_TOKEN) for word in text.split()] or [PAD_TOKEN] for text in texts]

# Collate function for DataLoader
def collate_fn(batch):
    inputs, targets = zip(*batch)
    inputs_padded = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in inputs], batch_first=True, padding_value=PAD_TOKEN)
    targets_padded = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in targets], batch_first=True, padding_value=PAD_TOKEN)
    return inputs_padded, targets_padded

# Dataset class
class ParagraphTitleDataset(Dataset):
    def __init__(self, input_sequences, target_sequences):
        self.input_sequences = input_sequences
        self.target_sequences = target_sequences
    def __len__(self):
        return len(self.input_sequences)
    def __getitem__(self, idx):
        return self.input_sequences[idx], self.target_sequences[idx]

# Define Encoder
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size, padding_idx=PAD_TOKEN)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
    def forward(self, input_seq):
        input_seq = torch.clamp(input_seq, min=0, max=self.embedding.num_embeddings - 1)
        embedded = self.dropout(self.embedding(input_seq))
        output, hidden = self.gru(embedded)
        hidden = hidden.view(2, hidden.size(1), -1).permute(1, 0, 2).contiguous().view(hidden.size(1), -1)
        return output, hidden

# Define Decoder with ReLU layer after embedding
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size, padding_idx=PAD_TOKEN)
        self.relu = nn.ReLU()  # Added ReLU layer
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)
    def forward(self, input_token, hidden):
        input_token = torch.clamp(input_token, min=0, max=self.embedding.num_embeddings - 1)
        embedded = self.embedding(input_token).unsqueeze(1)
        embedded = self.relu(embedded)  # Apply ReLU activation
        output, hidden = self.gru(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

# Define Seq2Seq Model
class Seq2SeqRNN(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqRNN, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.hidden_projection = nn.Linear(encoder.gru.hidden_size * 2, decoder.gru.hidden_size)
    def forward(self, input_seq, target_seq=None):
        batch_size = input_seq.size(0)
        encoder_output, encoder_hidden = self.encoder(input_seq)
        decoder_hidden = self.hidden_projection(encoder_hidden).unsqueeze(0)
        decoder_input = torch.full((batch_size, 1), SOS_TOKEN, device=input_seq.device, dtype=torch.long)
        if self.training and target_seq is not None:
            target_len = target_seq.size(1)
            outputs = torch.zeros(batch_size, target_len, self.decoder.out.out_features, device=input_seq.device)
            for t in range(target_len):
                decoder_output, decoder_hidden = self.decoder(decoder_input[:, -1], decoder_hidden)
                outputs[:, t, :] = decoder_output
                if t < target_len - 1:
                    decoder_input = torch.cat([decoder_input, target_seq[:, t].unsqueeze(1)], dim=1)
            return outputs
        else:
            outputs = []
            for _ in range(MAX_NEW_TOKENS):
                decoder_output, decoder_hidden = self.decoder(decoder_input[:, -1], decoder_hidden)
                outputs.append(decoder_output)
                top_token = decoder_output.argmax(dim=-1)
                decoder_input = torch.cat([decoder_input, top_token.unsqueeze(1)], dim=1)
                if torch.all(top_token == EOS_TOKEN):
                    break
            return torch.stack(outputs, dim=1)

# Training function
def train(model, data_loader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch_count, (input_seq, target_seq) in enumerate(data_loader, 1):
        input_seq, target_seq = input_seq.to(DEVICE), target_seq.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(input_seq, target_seq)
        outputs_flat = outputs.contiguous().view(-1, outputs.size(-1))
        target_flat = target_seq.contiguous().view(-1)

        # Fix vocabulary indexing issue
        max_vocab_index = max(vocabulary.values())
        vocab_size = max_vocab_index + 1
        max_target = target_flat.max().item()
        if max_target >= vocab_size:
            print(f"Batch {batch_count}: Target index {max_target} exceeds vocab size {vocab_size}")
            continue

        loss = criterion(outputs_flat, target_flat)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(data_loader)
    print(f"Training Loss: {avg_loss:.4f}")
    return avg_loss
# Evaluation function
def evaluate(model, data_loader, vocabulary):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_list = []

    with torch.no_grad():
        for input_seq, target_seq in data_loader:
            input_seq = input_seq.to(DEVICE)
            target_seq = target_seq.to(DEVICE)

            # Get model predictions
            outputs = model(input_seq)  # (batch_size, seq_len, vocab_size)

            # Get predicted tokens
            predicted_tokens = outputs.argmax(dim=-1)  # (batch_size, seq_len)

            # Convert tokens to text
            generated_texts = decode_outputs(predicted_tokens, vocabulary)
            reference_texts = decode_outputs(target_seq, vocabulary)

            # Calculate ROUGE scores
            for i in range(len(reference_texts)):
                scores_list.append(scorer.score(reference_texts[i], generated_texts[i]))

    return scores_list

# Decoding function
def decode_outputs(output_sequences, vocabulary):
    index_to_word = {idx: word for word, idx in vocabulary.items()}
    decoded_texts = []

    for sequence in output_sequences:
        words = []
        for idx in sequence:
            token = idx.item()
            if token == EOS_TOKEN:
                break
            if token != SOS_TOKEN and token != PAD_TOKEN:
                words.append(index_to_word.get(token, "<UNK>"))
        decoded_texts.append(" ".join(words))

    return decoded_texts

# Main execution
if __name__ == "__main__":
    import pandas as pd
    paragraphs_train_raw = df2[:-500].Text.tolist()
    titles_train_raw = df_title[:-500].title.tolist()

    vocabulary = create_vocabulary(paragraphs_train_raw, titles_train_raw, min_frequency=0.01)
    max_vocab_index = max(vocabulary.values())
    vocab_size = max_vocab_index + 1
    print(f"Vocabulary size: {vocab_size}")

    tokenized_paragraphs_train = tokenize_text(paragraphs_train_raw, vocabulary)
    tokenized_titles_train = tokenize_text(titles_train_raw, vocabulary)

    train_dataset = ParagraphTitleDataset(tokenized_paragraphs_train, tokenized_titles_train)
    train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, drop_last=True)

    encoder = EncoderRNN(vocab_size, HIDDEN_DIM).to(DEVICE)
    decoder = DecoderRNN(HIDDEN_DIM, vocab_size).to(DEVICE)
    seq2seq_model = Seq2SeqRNN(encoder, decoder).to(DEVICE)
    optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)

    for epoch in tqdm(range(3)):
        train(seq2seq_model, train_data_loader, optimizer, criterion)

    print("Training complete!")




In [ ]:
vocabulary

In [ ]:
# Updated Evaluation Function with title comparison
def evaluate(model, data_loader, vocabulary):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_list = []
    all_predictions = []
    all_references = []

    with torch.no_grad():
        for input_seq, target_seq in data_loader:
            input_seq = input_seq.to(DEVICE)
            target_seq = target_seq.to(DEVICE)

            # Get model predictions
            outputs = model(input_seq)  # (batch_size, seq_len, vocab_size)
            predicted_tokens = outputs.argmax(dim=-1)  # (batch_size, seq_len)

            # Decode predicted and reference tokens
            generated_texts = decode_outputs(predicted_tokens, vocabulary)
            reference_texts = decode_outputs(target_seq, vocabulary)

            # Calculate ROUGE scores and print comparisons
            for i in range(len(reference_texts)):
                score = scorer.score(reference_texts[i], generated_texts[i])
                scores_list.append(score)
                all_predictions.append(generated_texts[i])
                all_references.append(reference_texts[i])

                # Print actual vs. predicted
                # print(f"\nSample {len(scores_list)}:")
                # print(f"  🟩 Actual Title    : {reference_texts[i]}")
                # print(f"  🟦 Predicted Title : {generated_texts[i]}")
                # print(f"  📊 ROUGE Scores    : {score}")

    return scores_list, all_predictions, all_references


# Decoding function
def decode_outputs(output_sequences, vocabulary):
    index_to_word = {idx: word for word, idx in vocabulary.items()}
    decoded_texts = []

    for sequence in output_sequences:
        words = []
        for idx in sequence:
            token = idx.item()
            if token == EOS_TOKEN:
                break
            if token != SOS_TOKEN and token != PAD_TOKEN:
                words.append(index_to_word.get(token, "<UNK>"))
        decoded_texts.append(" ".join(words))

    return decoded_texts

# df_testing = pd.read_csv("/kaggle/input/nlp-assign/test_pre_corpus.csv")
# df_testing = pd.read_csv("/kaggle/input/nlp-assign/pre_corpus.csv")

# df_testing_title = pd.read_csv("/kaggle/input/nlp-assign/test(1).csv").title
test_paragraphs_raw = df2[-500:].Text.tolist()   # Replace with tokenized test paragraphs.
test_titles_raw = df_title[-500:].title.tolist()
# test_titles_raw = df_test_title.title.tolist()
# Step 1: Tokenize Test Data
# test_paragraphs_raw = ["Your test paragraph 1", "Your test paragraph 2", ...]  # Replace with actual test data
# test_titles_raw = ["Title 1", "Title 2", ...]  # Replace with actual titles

# Convert test data into tokenized format using the same vocabulary as training
tokenized_paragraphs_test = tokenize_text(test_paragraphs_raw, vocabulary)
tokenized_titles_test = tokenize_text(test_titles_raw, vocabulary)

# Step 2: Create Test Dataset and DataLoader
test_dataset = ParagraphTitleDataset(tokenized_paragraphs_test, tokenized_titles_test)
test_data_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)


rouge_scores, predicted_titles, actual_titles = evaluate(seq2seq_model, test_data_loader, vocabulary)


# # Step 4: Print Results
# for i, score in enumerate(rouge_scores):
#     print(f"Sample {i+1} ROUGE Scores:", score)

avg_rouge = {
    "rouge1": sum([s['rouge1'].fmeasure for s in rouge_scores]) / len(rouge_scores),
    "rouge2": sum([s['rouge2'].fmeasure for s in rouge_scores]) / len(rouge_scores),
    "rougeL": sum([s['rougeL'].fmeasure for s in rouge_scores]) / len(rouge_scores),
}

# Step 5: Print Average ROUGE Scores
print("Average ROUGE Scores:")
print(f"ROUGE-1: {avg_rouge['rouge1']:.4f}")
print(f"ROUGE-2: {avg_rouge['rouge2']:.4f}")
print(f"ROUGE-L: {avg_rouge['rougeL']:.4f}")

# B2

In [ ]:


def collate_fn(batch):
    inputs, targets = zip(*batch)

    # Pad input sequences
    inputs_padded = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in inputs], batch_first=True, padding_value=PAD_TOKEN)

    # Pad target sequences (ensure target sequences are padded the same way)
    targets_padded = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in targets], batch_first=True, padding_value=PAD_TOKEN)

    return inputs_padded, targets_padded



In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from rouge_score import rouge_scorer
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

def load_glove_embeddings(embedding_path, vocabulary, embedding_dim=300):
    """Loads pre-trained GloVe embeddings and returns an embedding matrix."""
    embeddings_index = {}

    with open(embedding_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = vector

    embedding_matrix = np.random.normal(scale=0.6, size=(len(vocabulary), embedding_dim))
    for word, idx in vocabulary.items():
       if idx < vocab_size:  # Ensure index is within bounds
        if word in embeddings_index:
            embedding_matrix[idx] = embeddings_index[word]

    return torch.tensor(embedding_matrix, dtype=torch.float)


In [ ]:
class HierEncoderRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super(HierEncoderRNN, self).__init__()
        # Embedding layer with padding_idx to ignore padding tokens during the embedding lookup
        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=PAD_TOKEN)

        # Word-level GRU to process words within each sentence
        self.word_gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

        # Sentence-level GRU to process the representation of sentences
        self.sent_gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

        self.dropout = nn.Dropout(0.3)

    def forward(self, input_seq):
        # Apply embedding to input sequence
        embedded = self.dropout(self.embedding(input_seq))

        # Process words within each sentence using GRU
        word_output, word_hidden = self.word_gru(embedded)

        # Averaging word embeddings per sentence
        sentence_embeddings = word_output.mean(dim=1, keepdim=True)

        # Process sentence embeddings using sentence-level GRU
        sent_output, sent_hidden = self.sent_gru(sentence_embeddings)

        return sent_output, sent_hidden



class Decoder2RNN(nn.Module):
    def __init__(self, hidden_size, output_size, inference=False):
        super(Decoder2RNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size, padding_idx=PAD_TOKEN)
        self.gru1 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.gru2 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.inference = inference

    def forward(self, input_token, hidden, inference=False):
        # Handling input size for training vs inference
        if inference:
            # Make sure input_token is 2D for embedding lookup
            if input_token.dim() == 0:  # Scalar tensor
                input_token = input_token.unsqueeze(0)

            # Create proper embedding with batch dimension
            embedded = self.embedding(input_token)

            # Make sure embedded has 3 dimensions [batch_size, seq_len, hidden_size]
            if embedded.dim() == 2:
                embedded = embedded.unsqueeze(1)  # Add sequence length dimension
        else:
            embedded = self.embedding(input_token).unsqueeze(1)  # Shape: (batch_size, 1, hidden_size)

        # GRU operations
        output, hidden1 = self.gru1(embedded, hidden)
        output, hidden2 = self.gru2(output, hidden1)
        output = self.softmax(self.out(output.squeeze(1)))  # Remove the second dimension (1) after GRU
        return output, hidden2



In [ ]:
import torch
import torch.nn as nn
import numpy as np
import heapq

class Seq2SeqRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_path=None,
                 use_hierarchical=False, use_decoder2=False, use_beam_search=False, beam_width=3):
        """
        A flexible Seq2Seq model with optional:
        - GloVe Embeddings
        - Hierarchical Encoding
        - Double GRU Decoder
        - Beam Search
        """
        super(Seq2SeqRNN, self).__init__()

        # Encoder selection
        self.encoder = HierEncoderRNN(vocab_size, hidden_size) if use_hierarchical else EncoderRNN(vocab_size, hidden_size)

        # Decoder selection
        self.decoder = Decoder2RNN(hidden_size, vocab_size) if use_decoder2 else DecoderRNN(hidden_size, vocab_size)

        # Beam Search
        self.use_beam_search = use_beam_search
        self.beam_width = beam_width

         # Set inference_mode to False by default (indicating training mode)
        # self.inference_mode = False

        # Load GloVe embeddings if provided
        if embedding_path:
            self.load_embeddings(embedding_path)

    def load_embeddings(self, embedding_path):
        """Loads GloVe embeddings into the encoder's embedding layer."""
        embedding_matrix = load_glove_embeddings(embedding_path, vocabulary)
        self.encoder.embedding.weight.data.copy_(embedding_matrix)
        self.encoder.embedding.weight.requires_grad = False  # Freeze embeddings

    def forward(self, input_seq, target_seq=None):
        input_seq = input_seq.to(DEVICE)
        encoder_output, encoder_hidden = self.encoder(input_seq)
        decoder_hidden = encoder_hidden

        if self.use_beam_search and target_seq is None:
            return self.beam_search_decoder(input_seq, decoder_hidden)

        # Teacher forcing during training
        decoder_input = torch.full((input_seq.size(0), 1), SOS_TOKEN, device=DEVICE)
        outputs = []

        for i in range(MAX_NEW_TOKENS):
            decoder_output, decoder_hidden = self.decoder(decoder_input[:, -1], decoder_hidden)
            outputs.append(decoder_output)
            top_token = decoder_output.argmax(dim=1)
            decoder_input = torch.cat([decoder_input, top_token.unsqueeze(1)], dim=1)

            # Teacher forcing: Use target word if provided
            if target_seq is not None and i < target_seq.shape[1]:
                decoder_input[:, -1] = target_seq[:, i]

            if torch.all(top_token == EOS_TOKEN):
                break

        outputs = torch.stack(outputs).transpose(0, 1)  # Shape: (batch_size, seq_len, vocab_size)

        # Ensure that the target sequence and output match in shape
        max_len = target_seq.size(1) if target_seq is not None else outputs.size(1)
        outputs = outputs[:, :max_len, :]  # Truncate if necessary to match target sequence length

        return outputs


    def beam_search_decoder(self, input_seq, decoder_hidden, inference=True):
        """Performs Beam Search decoding."""
        batch_size = input_seq.size(0)
        all_results = []

        # Process each item in the batch separately
        for b in range(batch_size):
            # Extract the hidden state for this batch item only
            # Original shape: [layers, batch_size, hidden_dim]
            # We want: [layers, 1, hidden_dim]
            single_hidden = decoder_hidden.clone()
            single_hidden = single_hidden[:, b:b+1, :]  # Keep dimension with size 1

            beams = [(0.0, [SOS_TOKEN], single_hidden)]  # (score, sequence, hidden_state)

            for _ in range(MAX_NEW_TOKENS):
                new_beams = []
                for score, seq, hidden in beams:
                    if seq[-1] == EOS_TOKEN:
                        new_beams.append((score, seq, hidden))
                        continue

                    # Decoder input for the next token in the sequence
                    decoder_input = torch.tensor([seq[-1]], device=DEVICE)

                    # Pass through decoder
                    decoder_output, new_hidden = self.decoder(decoder_input, hidden, inference=inference)

                    # Get the top-k probabilities and indices
                    topk_probs, topk_indices = torch.topk(decoder_output, self.beam_width)

                    # Generate new beams with new sequences
                    for i in range(self.beam_width):
                        new_seq = seq + [topk_indices[0][i].item()]
                        new_score = score + topk_probs[0][i].item()
                        new_beams.append((new_score, new_seq, new_hidden))

                # Sort beams by score and keep the top-k
                beams = sorted(new_beams, key=lambda x: x[0], reverse=True)[:self.beam_width]

                # Stop if all top beams end with EOS_TOKEN
                if all(seq[-1] == EOS_TOKEN for _, seq, _ in beams[:min(3, len(beams))]):
                    break

            # Get best sequence for this batch item
            best_seq = beams[0][1]
            all_results.append(best_seq)

        # Pad sequences to the same length
        max_length = max(len(seq) for seq in all_results)
        padded_results = [seq + [PAD_TOKEN] * (max_length - len(seq)) for seq in all_results]

        # Convert to tensor
        return torch.tensor(padded_results, device=DEVICE)

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader

# ====== 1. Load Data ======
# Replace this with actual dataset
paragraphs_train_raw = df2[:-500].Text.tolist()  # Load training data
titles_train_raw = df_title[:-500].title.tolist()  # Load corresponding titles

paragraphs_test_raw = df2[-500:].Text.tolist()  # Load test data
titles_test_raw = df_title[-500:].title.tolist()  # Load corresponding test titles

# ====== 2. Create Vocabulary ======
vocabulary = create_vocabulary(paragraphs_train_raw,titles_train_raw, min_frequency=0.01)
vocab_size = len(vocabulary)

# Tokenizing Data
tokenized_paragraphs_train = tokenize_text(paragraphs_train_raw, vocabulary)
tokenized_titles_train = tokenize_text(titles_train_raw, vocabulary)

tokenized_paragraphs_test = tokenize_text(paragraphs_test_raw, vocabulary)
tokenized_titles_test = tokenize_text(titles_test_raw, vocabulary)

# Create Datasets & DataLoaders
train_dataset = ParagraphTitleDataset(tokenized_paragraphs_train, tokenized_titles_train)
test_dataset = ParagraphTitleDataset(tokenized_paragraphs_test, tokenized_titles_test)

# train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn, drop_last=True)

test_data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

# ====== 3. Initialize Model ======
seq2seq_model = Seq2SeqRNN(
    vocab_size=vocab_size,
    hidden_size=HIDDEN_DIM,
    embedding_path="/kaggle/input/glove-data/glove.6B.300d.txt",  # Provide GloVe path if available
    use_hierarchical=True,  # Use hierarchical encoder
    use_decoder2=True,  # Use dual-GRU decoder
    use_beam_search=True,  # Enable beam search
    beam_width=5  # Set beam width
).to(DEVICE)

optimizer = optim.Adam(seq2seq_model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)  # Ignore padding during loss calculation

# ====== 4. Training Function ======
def train(model, data_loader, optimizer, criterion, num_epochs=3):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for input_seq, target_seq in data_loader:
            input_seq, target_seq = input_seq.to(DEVICE), target_seq.to(DEVICE)

            # Ensure the target sequence and output match in length
            max_len = max(input_seq.size(1), target_seq.size(1))
            target_seq = target_seq[:, :max_len]  # Trim target sequence if necessary

            optimizer.zero_grad()

            # Model forward pass
            outputs = model(input_seq, target_seq)

            # print(outputs.size())
            # print(target_seq.size())
            # Ensure the output and target are the same shape
            outputs = outputs.reshape(-1, outputs.size(-1))  # Flatten the output for loss calculation
            target_seq = target_seq.view(-1)


            # Apply the CrossEntropy loss while ignoring padding
            loss = criterion(outputs, target_seq)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(data_loader)
        print(f"Epoch {epoch+1}, Training Loss: {avg_loss:.4f}")


# ====== 5. Evaluation Function ======
from rouge_score import rouge_scorer

# def evaluate(model, data_loader, vocabulary):
#     model.eval()
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
#     scores_list = []

#     with torch.no_grad():
#         for input_seq, target_seq in data_loader:
#             input_seq = input_seq.to(DEVICE)
#             target_seq = target_seq.to(DEVICE)

#             # Get model predictions
#             outputs = model(input_seq)  # This should now be a tensor from beam_search_decoder

#             # If the outputs are a list (from beam search), stack them into a tensor
#             if isinstance(outputs, list):
#                 outputs = torch.stack(outputs, dim=1)  # Stack the list into a 2D tensor

#             # Ensure outputs are 2D: (batch_size, seq_len)
#             if outputs.dim() == 1:  # If the output is 1D (e.g., from beam search)
#                 outputs = outputs.unsqueeze(0)  # Make it 2D with shape [1, seq_len]

#             # Perform argmax and slicing for sequence length
#             generated_texts = decode_outputs(outputs.argmax(dim=-1)[:, :target_seq.size(1)], vocabulary)
#             # generated_texts = decode_outputs(outputs.argmax(dim=-1), vocabulary)
#             # generated_texts = decode_outputs(outputs.argmax(dim=-1), vocabulary)
#             reference_texts = decode_outputs(target_seq, vocabulary)

#             # Calculate ROUGE scores
#             for i in range(len(reference_texts)):
#                 scores_list.append(scorer.score(reference_texts[i], generated_texts[i]))

#     avg_rouge = {
#         "rouge1": sum([s['rouge1'].fmeasure for s in scores_list]) / len(scores_list),
#         "rouge2": sum([s['rouge2'].fmeasure for s in scores_list]) / len(scores_list),
#         "rougeL": sum([s['rougeL'].fmeasure for s in scores_list]) / len(scores_list),
#     }

#     return avg_rouge

def evaluate(model, data_loader, vocabulary):
    model.eval()
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores_list = []

    with torch.no_grad():
        for input_seq, target_seq in data_loader:
            input_seq = input_seq.to(DEVICE)
            target_seq = target_seq.to(DEVICE)

            # Get model predictions
            outputs = model(input_seq)  # This calls beam_search_decoder if needed

            # Check dimensionality of outputs and handle accordingly
            if outputs.dim() == 1:
                # If outputs is 1D, it's a single sequence (convert to 2D with batch_size=1)
                outputs = outputs.unsqueeze(0)

            # Ensure outputs are the right shape for decoding
            if outputs.dim() == 2:
                # If no vocabulary dimension (just batch x sequence_length), assume these are indices
                # No need for argmax - these are already the best token indices
                generated_indices = outputs
            else:
                # Normal case - outputs are [batch, seq_len, vocab_size]
                generated_indices = outputs.argmax(dim=-1)

            # Slice to match target length if needed
            if target_seq.size(1) < generated_indices.size(1):
                generated_indices = generated_indices[:, :target_seq.size(1)]

            generated_texts = decode_outputs(generated_indices, vocabulary)
            reference_texts = decode_outputs(target_seq, vocabulary)

            # Calculate ROUGE scores
            for i in range(len(reference_texts)):
                scores_list.append(scorer.score(reference_texts[i], generated_texts[i]))

    avg_rouge = {
        "rouge1": sum([s['rouge1'].fmeasure for s in scores_list]) / len(scores_list),
        "rouge2": sum([s['rouge2'].fmeasure for s in scores_list]) / len(scores_list),
        "rougeL": sum([s['rougeL'].fmeasure for s in scores_list]) / len(scores_list),
    }

    return avg_rouge


# ====== 6. Run Training ======
print("Starting Training...")
train(seq2seq_model, train_data_loader, optimizer, criterion)
print("Training Completed!")

# ====== 7. Run Evaluation ======
print("Evaluating Model...")
rouge_scores = evaluate(seq2seq_model, test_data_loader, vocabulary)
print("ROUGE Scores:", rouge_scores)


Vocabulary size: 9215
Starting Training...
Epoch 1, Training Loss: 4.7510
Epoch 2, Training Loss: 3.5173
Epoch 3, Training Loss: 2.8513
Training Completed!
Evaluating Model...
ROUGE Scores: {'rouge1': 0.36812265512265546, 'rouge2': 0.08726984126984123, 'rougeL': 0.36812265512265546}


# C1

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoTokenizer
import pandas as pd
import torch
from datasets import Dataset, DatasetDict

# ====== 1. Load Datasets from CSV ======
train_df = pd.read_csv("/kaggle/input/nlp-assign/train(1).csv")[:-500]  # Ensure train.csv has 'text' and 'title' columns
val_df = pd.read_csv("/kaggle/input/nlp-assign/train(1).csv")[-500:]      # Ensure val.csv has 'text' and 'title' columns
test_df = pd.read_csv("/kaggle/input/nlp-assign/test(1).csv")    # Ensure test.csv has 'text' column (no titles)

# ====== 2. Load Tokenizer ======
model_name = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ====== 3. Preprocessing Function ======
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# ====== 4. Convert DataFrames to HuggingFace Dataset ======
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Wrap datasets in DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# ====== 5. Load Pretrained T5 Model ======
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# ====== 6. Define Training Arguments ======
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True
)

# ====== 7. Trainer Setup ======
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer
)

# ====== 8. Train the Model ======
trainer.train()




In [ ]:
def generate_title_beam(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to("cuda")
    with torch.no_grad():
        output = model.generate(**inputs, num_beams=5, early_stopping=True)  # Beam search with 5 beams
    return tokenizer.decode(output[0], skip_special_tokens=True)


# Generate Titles for Test Set
test_df["generated_title"] = test_df["text"].apply(generate_title_beam)
test_df.to_csv("test_predictions_beam.csv", index=False)

print("Predictions saved to test_predictions_beam.csv")

In [ ]:
import pandas as pd
from rouge_score import rouge_scorer

# Load the test predictions
# df = pd.read_csv("test_predictions.csv")  # Ensure this file has 'title' & 'generated_title' columns
df = test_df
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Function to compute ROUGE scores
def compute_rouge(df):
    rouge1, rouge2, rougeL = [], [], []

    for ref, gen in zip(df["title"], df["generated_title"]):
        scores = scorer.score(str(ref), str(gen))  # Convert to string in case of NaN values
        rouge1.append(scores['rouge1'].fmeasure)
        rouge2.append(scores['rouge2'].fmeasure)
        rougeL.append(scores['rougeL'].fmeasure)

    return {
        "ROUGE-1": sum(rouge1) / len(rouge1),
        "ROUGE-2": sum(rouge2) / len(rouge2),
        "ROUGE-L": sum(rougeL) / len(rougeL)
    }

# Compute ROUGE scores
rouge_scores = compute_rouge(df)
print("ROUGE Scores:", rouge_scores)
